# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [32]:
from pymongo import MongoClient

import warnings
warnings.filterwarnings('ignore')

In [33]:
cliente = MongoClient('mongodb://localhost:27017')  

cliente

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [34]:
cliente.list_database_names()

['admin', 'companies', 'config', 'local']

In [46]:
client = MongoClient()
db = client.companies  
colec = db.companies  

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [47]:
list(colec.find({'name':'Babelgum'},{'name': 1,'_id': 0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [64]:
list(colec.find({'number_of_employees':{'$gte': 5000}},{'name': 1}).sort('number_of_employees', 1).limit( 20))

[{'_id': ObjectId('52cdef7e4bab8bd67529aa77'),
  'name': 'JDS Uniphase Corporation'},
 {'_id': ObjectId('52cdef7f4bab8bd67529c4bf'), 'name': 'United Internet'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b1b5'), 'name': 'Air Astana Airlines'},
 {'_id': ObjectId('52cdef7f4bab8bd67529c5d2'), 'name': 'Vcare Call Center'},
 {'_id': ObjectId('52cdef7e4bab8bd67529a4f4'), 'name': 'McAfee'},
 {'_id': ObjectId('52cdef7d4bab8bd67529891b'), 'name': 'Spotify'},
 {'_id': ObjectId('52cdef7d4bab8bd675299e01'), 'name': 'PCH International'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e49'), 'name': 'Nintendo'},
 {'_id': ObjectId('52cdef7e4bab8bd67529ba2e'),
  'name': 'Hexaware Technologies'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'), 'name': 'Facebook'},
 {'_id': ObjectId('52cdef7e4bab8bd67529a453'), 'name': 'OpenText'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b9dd'), 'name': 'CPM Braxis'},
 {'_id': ObjectId('52cdef7d4bab8bd675298b89'), 'name': 'LSI'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b691'),
  'nam

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [65]:
query = {"$and":[{"founded_year":{"$gte": 2000}},{"founded_year":{"$lte": 2005}}]}
list(db["companies"].find(query,{"_id": 0,"name": 1,"founded_year": 1}).limit(20))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [66]:
query = {"$and":[{"ipo.valuation_amount":{"$gte": 100000000}}, {"founded_year":{'$lte': 2010}}]}
list(db["companies"].find(query,{"_id": 0,"name": 1,"ipo.valuation_amount": 1}).limit(20))

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Brightcove', 'ipo': {'valuation_amount': 290000000}},
 {'name': 'KIT digital', 'ipo': {'valuation_amount': 235000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'OpenTable', 'ipo': {'valuation_amount': 1050000000}},
 {'name': 'ChannelAdvisor', 'ipo': {'valuation_amount': 287000000}},
 {'name': 'Jive Software', 'ipo': {'valuation_amount': 1000000000}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000}},
 {'name': 'Wix', 'ipo': {'valuation_amount': 750000000}},
 {'name': 'Shutterfly', 'ipo': {'valuation_amount': 350000000}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000}},
 {'name': 'Salesforce', 'ipo': {'

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [67]:
query = {"$and":[{"number_of_employees":{"$lte": 1000}}, {"founded_year":{"$lte": 2005}}]}
list(db["companies"].find(query,{"_id": 0,"name": 1,"number_of_employees": 1, "founded_year": 1}).sort("number_of_employees", -1).limit(10))

[{'name': 'Omniture', 'number_of_employees': 1000, 'founded_year': 1996},
 {'name': 'Telenav', 'number_of_employees': 1000, 'founded_year': 1999},
 {'name': 'Gumtree', 'number_of_employees': 1000, 'founded_year': 2005},
 {'name': 'SK Net Service Company Ltd',
  'number_of_employees': 1000,
  'founded_year': 2001},
 {'name': 'Yodle', 'number_of_employees': 1000, 'founded_year': 2005},
 {'name': "OC'LIANE", 'number_of_employees': 1000, 'founded_year': 2003},
 {'name': 'FANUC Robotics America',
  'number_of_employees': 1000,
  'founded_year': 1982},
 {'name': 'Antal International',
  'number_of_employees': 1000,
  'founded_year': 1993},
 {'name': 'Sonus Networks', 'number_of_employees': 1000, 'founded_year': 1997},
 {'name': 'Akamai Technologies',
  'number_of_employees': 1000,
  'founded_year': 1998}]

### 6. All the companies that don't include the `partners` field.

In [71]:
list(colec.find({'partners':{'$ne': []}},{'name': 1}).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297dba'), 'name': 'Google'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e39'), 'name': 'Parakey'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e7e'), 'name': 'boo-box'},
 {'_id': ObjectId('52cdef7c4bab8bd675297eb0'), 'name': 'Echo'},
 {'_id': ObjectId('52cdef7c4bab8bd675297eb4'), 'name': 'Pickle'},
 {'_id': ObjectId('52cdef7c4bab8bd675297ed3'), 'name': 'Weebly'},
 {'_id': ObjectId('52cdef7c4bab8bd675297ed4'), 'name': 'Yola'},
 {'_id': ObjectId('52cdef7c4bab8bd675297f00'), 'name': 'Docstoc'},
 {'_id': ObjectId('52cdef7c4bab8bd675297f0c'), 'name': 'Alibaba'},
 {'_id': ObjectId('52cdef7c4bab8bd675297f24'), 'name': 'TubeMogul'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [72]:
query = {"category_code": { "$type": "null" }}
list(db["companies"].find(query,{"_id": 0,"name": 1,"category_code": 1}).limit(10))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None},
 {'name': 'Drigg', 'category_code': None},
 {'name': 'SpaceTime', 'category_code': None},
 {'name': 'Touch Clarity', 'category_code': None},
 {'name': 'MMDAYS', 'category_code': None},
 {'name': 'Inside Group', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [73]:
query = {"$and":[{"number_of_employees":{"$gte": 100}}, {"number_of_employees":{"$lte": 1000}}]}
list(db["companies"].find(query,{"_id": 0,"name": 1,"number_of_employees": 1}).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [74]:
query = {"ipo.valuation_amount":{"$gte": 0}}
list(db["companies"].find(query,{"_id": 0,"name": 1,"ipo.valuation_amount": 1}).sort("ipo.valuation_amount", -1).limit(10))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [75]:
query = {"number_of_employees":{"$gte": 0}}
list(db["companies"].find(query,{"_id": 0,"name": 1,"number_of_employees": 1}).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [120]:
query = {'founded_month': {'$gte': 7}}
list(db["companies"].find(query,{'_id': 0,'name': 1}).limit(10)) # pongo 10 en lugar de 1000 para que no scrolees

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Omnidrive'},
 {'name': 'eBay'},
 {'name': 'Joost'},
 {'name': 'Plaxo'},
 {'name': 'Powerset'},
 {'name': 'Kyte'},
 {'name': 'Thoof'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [119]:
query = {"$and":[{'founded_year': {'$lt': 2000}},{'acquisition.price_amount': {'$gt': 10000000}}]}
list(db["companies"].find(query, {'_id': 0, 'name': 1}).limit(10))

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'},
 {'name': 'Neopets'},
 {'name': 'Sun Microsystems'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Sabre'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [118]:
query = {'acquisition.acquired_year': {'$gt': 2010}}
list(db["companies"].find(query, {'_id': 0, 'name': 1, 'acquisition':1}).sort('acquisition.price_amount',-1).limit(3))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name': 'LS

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [117]:
list(colec.find({}, {'_id': 0, 'founded_year': 1, 'name': 1}).sort('founded_year', -1).limit(20))

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Shopseen', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'VistaGen Therapeutics', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Tongxue', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Skydeck', 'founded_year': 2012},
 {'name': 'Springleap', 'founded_year': 2012},
 {'name': 'Mobiluck', 'founded_year': 2012},
 {'name': 'Jumbuck Entertainment', 'founded_year': 2012},
 {'name': 'Bling Easy', 'founded_year': 2012},
 {'name': 'PeekYou', 'founded_year': 2012},
 {'name': 'Simplicant', 'founded_year': 2012}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [116]:
query = {'founded_day': {'$lte': 7}}
list(db["companies"].find(query, {'_id': 0,'name': 1,'founded_day': 1}).sort('acquisition.price_amount', -1).limit(10))

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'Vitrue', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [115]:
query = {'$and': [{'category_code':'web'}, {'number_of_employees': {'$gt': 4000}}]}
list(db["companies"].find(query, {'_id': 0,'name': 1,'number_of_employees': 1}).sort('number_of_employees', 1).limit(10))

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [114]:
query = {'$and': [{'acquisition.price_currency_code':'EUR'},{'acquisition.price_amount': {'$gt': 10000000}}]}
list(db["companies"].find(query, {'_id': 0, 'name': 1}).limit(10))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [103]:
query = {'acquisition.acquired_month': {'$lte': 3}}
list(db["companies"].find(query, {'_id': 0,'name': 1,'acquisition':1}).limit(3)) # es un limit 10, pero ocupa demasiado 

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [113]:
query =  {'$and': [{'founded_year': {'$gt': 2000}}, {'founded_year': {'$lt': 2010}},{'acquisition.acquired_year': {'$gt': 2011}}]}
list(db["companies"].find(query, {'_id': 0,'name': 1}).limit(10))

[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'blogTV'},
 {'name': 'Revision3'},
 {'name': 'iContact'},
 {'name': 'Mashery'},
 {'name': 'Dailymotion'},
 {'name': 'KickApps'},
 {'name': 'Netvibes'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [121]:
query = {'deadpooled_year': {'$gt': 3}}
list(db["companies"].find(query, {'_id': 0,'name': 1}).limit(10))              

[{'name': 'Omnidrive'},
 {'name': 'Babelgum'},
 {'name': 'Sparter'},
 {'name': 'Thoof'},
 {'name': 'Mercora'},
 {'name': 'Wesabe'},
 {'name': 'Jangl SMS'},
 {'name': 'Stickam'},
 {'name': 'AllPeers'},
 {'name': 'EQO'}]